## **DAY 7- Workflows & Job Orchestration**

In [0]:
dbutils.widgets.text("source_path", "/default/path")


In [0]:
dbutils.widgets.dropdown(
    "layer",
    "bronze",
    ["bronze","silver","gold"]
)


In [0]:
source = dbutils.widgets.get("source_path")
layer = dbutils.widgets.get("layer")


In [0]:
from pyspark.sql.functions import count
def run_layer(layer_name):
    if layer_name == "bronze":
        print("Running BRONZE layer")
        # bronze logic 

        bronze_df = spark.read.csv(
        "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
        header=True,
        inferSchema=True)

        bronze_df.write \
        .format("delta") \
        .mode("overwrite") \
        .save("/Volumes/workspace/ecommerce/ecommerce_data/delta/bronze/events")

    elif layer_name == "silver":
        print("Running SILVER layer")
        # silver logic 
        silver_df = spark.read.format("delta").load(
        "/Volumes/workspace/ecommerce/ecommerce_data/delta/bronze/events")

        silver_clean = silver_df \
            .dropDuplicates(["event_time", "user_id"]) \
            .dropna(subset=["event_time", "user_id", "event_type"])

        silver_clean.write \
        .format("delta") \
        .mode("overwrite") \
        .save("/Volumes/workspace/ecommerce/ecommerce_data/delta/silver/events")



    elif layer_name == "gold":
        print("Running GOLD layer")
        # gold logic 
        gold_df = spark.read.format("delta").load(
        "/Volumes/workspace/ecommerce/ecommerce_data/delta/silver/events")

        gold_agg = gold_df.groupBy("event_type").agg(
            count("*").alias("total_events"))
        
        gold_agg.write \
        .format("delta") \
        .mode("overwrite") \
        .save("/Volumes/workspace/ecommerce/ecommerce_data/delta/gold/event_summary")

    else:
        print("Invalid layer name")

run_layer(layer)

Running BRONZE layer
